In [2]:
import os
import yaml
from pathlib import Path
from ultralytics import YOLO
import torch
from datetime import datetime

In [ ]:

class DentalDiagnosis:
    
    def __init__(self, project_name="dental_segmentation"):
        self.project_name = project_name
        self.base_dir = Path(__file__).parent
        self.setup_directories()
        
        
    def setup_directories(self):
        """create necessary directories"""
        dirs = [
            'data',
            'models',
            'results',
            'configs',
            'logs'
        ]
        for dir_name in dirs:
            (self.base_dir / dir_name).mkdir(exist_ok=True)
            

      
    def train_model(self,
                   model_name='yolov8m-seg',
                   data_yaml='configs/data.yaml',
                   epochs=100,
                   imgsz=640,
                   batch=16,
                   device='0',
                   patience=50,
                   save_period=10,
                   **kwargs):
        """
        YOLO modelini eğit
        
        Args:
            model_name: Kullanılacak YOLO modeli
            data_yaml: Data config dosyası yolu
            epochs: Epoch sayısı
            imgsz: Görüntü boyutu
            batch: Batch boyutu
            device: GPU device ('0', 'cpu', veya '0,1,2,3')
            patience: Early stopping patience
            save_period: Model kaydetme periyodu
            **kwargs: Ek eğitim parametreleri
        """
        
        
        model = YOLO(model=model_name)
        
        # farklı farklı çalıştırmalar için benzersiz klasörser oluşturur
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        project_dir = f"results/{self.project_name}_{model_name}"
        run_name = f"run_{timestamp}"
        
        print(f"model adı: {model_name}")
        print(f"yaml config name: {data_yaml}")
        print(f"Image size: {imgsz}")
        print(f"Batch size: {batch}")
        print(f"Epochs: {epochs}")
        print(f"Results: {project_dir}/{run_name}")

        print(f"{'='*60}\n")
        
        # Eğitim parametreleri
        train_args = {
            'data': data_yaml,
            'epochs': epochs,
            'imgsz': imgsz,
            'batch': batch,
            'device': device,
            'project': project_dir,
            'name': run_name,
            'patience': patience,
            'save': True,
            'save_period': save_period,
            'cache': True,
            'plots': True,
            'verbose': True,
            'amp': True,
            'resume': kwargs.get('resume', True),
            'lr0': 0.001,
            'lrf': 0.01,
            'optimizer': 'AdamW'
            **kwargs
        }
        
        results = model.train(**train_args)
        
        print(f"\nEğitim tamamlandı!")
        print(f"📁 Model kaydedildi: {project_dir}/{run_name}/weights/best.pt")
        
        return results
    
    def validate_model(self, 
                      model_path,
                      data_yaml='configs/data.yaml',
                      imgsz=640,
                      batch=16,
                      device='0'):
        

        print(f"validation starts...")
        
        model = YOLO(model_path)
        metrics = model.val(
            data=data_yaml,
            imgsz=imgsz,
            batch=batch,
            device=device,
            plots=True
        )
        
        print(f"\nvalidation successful")
        return metrics
    
    def predict(self,
               model_path,
               source,
               save=True,
               conf=0.25,
               iou=0.7,
               imgsz=640,
               device='0',
               show_labels=True,
               show_conf=True):
        """
        Tahmin yap
        
        Args:
            model_path: Eğitilmiş model yolu
            source: Tahmin yapılacak kaynak (resim, klasör, video)
            save: Sonuçları kaydet
            conf: Confidence threshold
            iou: IoU threshold
            imgsz: Görüntü boyutu
            device: GPU device
            show_labels: Etiketleri göster
            show_conf: Confidence skorunu göster
        """
        
        model = YOLO(model_path)
        
        results = model.predict(
            source=source,
            save=save,
            conf=conf,
            iou=iou,
            imgsz=imgsz,
            device=device,
            show_labels=show_labels,
            show_conf=show_conf,
            project='results/predictions',
            name='predict_' + datetime.now().strftime("%Y%m%d_%H%M%S")
        )
        
        return results
    
    def export_model(self, 
                    model_path,
                    format='onnx',
                    imgsz=640):
        """
        Modeli farklı formatlara export et
        
        Desteklenen formatlar: onnx, torchscript, tensorflow, tflite, etc.
        """
        
        model = YOLO(model_path)
        export_path = model.export(format=format, imgsz=imgsz)
        
        print(f"Model export edildi: {export_path}")
        return export_path

In [15]:
project = DentalDiagnosis("dental_lesion")

In [ ]:
project.train_model(
    model_name='yolov8x-seg',
    data_yaml='./data.yaml',
    epochs=3,
    imgsz=1248,
    batch=8,
    patience=20,
    save_period=5,
    project_name='data_v2_aug_base',

)

model adı: yolov8m-seg
yaml config name: /Users/sahindogruca/Desktop/code/ytü/semester-7/computerProject2/configs/yolov8-v1-test.yaml
Image size: 640
Batch size: 16
Epochs: 3
Results: results/dental_lesion_yolov8m-seg/run_20251107_215529

Ultralytics 8.3.226 🚀 Python-3.10.16 torch-2.9.0 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: None
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.
